In [ ]:
# default_exp colab

# colab
> helpers for colab

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from nicHelper.encrypt import encryptPasswordAes, decryptPasswordAes
from beartype import beartype
from getpass import getpass
from functools import partial
from typing import Tuple, Optional
from getpass import getpass
from pathlib import Path
import subprocess, pickle, os

# save aws pw

In [ ]:
#export
@beartype
def saveAwsPw(awskey:bytes, awssecret:bytes,
              path:str = '/content/drive/MyDrive/.awskey', password:bytes = b''):
  '''
    save your aws password in your 
  '''

  # encrypt with aes
  encrypt = partial(encryptPasswordAes, password)
  cryptkey, keynonce = encrypt(awskey)
  cryptsecret, secretnonce = encrypt(awssecret)
  
  # save file as pickle
  print(f'saving the AES encrypted credentials to {path}')
  with open (path ,'wb') as f:
    pickle.dump((cryptkey, cryptsecret, keynonce, secretnonce),f)
  

In [ ]:
def test_saveAwsPw():
  # enter password == test ##
  saveAwsPw(awskey = b'test', awssecret=b'password', path = '/tmp/test', password = b'test')
  with open ('/tmp/test', 'rb') as f:
    r = pickle.load(f)
  print(r)
test_saveAwsPw()

saving the AES encrypted credentials to /tmp/test
(b'\xe3\xd5\xd1#', b'&\xd5\xd6b\xd6+[j', b"'\x0e.\x99M;\\\x9f`$=v\xf0\x8b\x19\xec", b'`\xf8+\xd3jd8\xc5\x9a\x7fK\x93\xb8esm')


# load aws pw

In [ ]:
#export
@beartype
def loadAwsPw(path = '/content/drive/MyDrive/.awskey', 
              password:Optional[bytes] = b'' )->Optional[Tuple[bytes,bytes]]:
  '''
    load key and secret from path
  '''
  print( f'loading password from {path}')
  
  with open (path, 'rb') as f:
    cryptkey, cryptsecret, keynonce, secretnonce = pickle.load(f)
  decryptedKey = decryptPasswordAes(cryptkey,password,keynonce)
  decryptedSecret = decryptPasswordAes(cryptsecret, password, secretnonce)
  return decryptedKey, decryptedSecret
  

In [ ]:
def test_loadAwsPw():
  r = loadAwsPw(path = '/tmp/test', password = b'test')
  print(r)
test_loadAwsPw()

loading password from /tmp/test
(b'test', b'password')


# set up aws

In [ ]:
#export
@beartype
def setUpAws(awsKey:str, awsSecret:str, 
             profile:Optional[str] = None, region:str = 'ap-southeast-1')->Tuple[str,str,str]:
  ''' 
    this code generate a string to be executed to the shell
    you can do this with, however, its not a bad idea to check before executing
    
    (exec(c) for c in setUpAws(...))
    
    response:
      (setupKey, setupSecret, setupRegion)
      
  '''
  if profile:  profileParameter = f'--profile {profile} '
  else: profileParameter = ''
    
  setupPrefix = f'aws configure {profileParameter}'
  setupKey = f'{setupPrefix}set aws_access_key_id {awsKey}'
  setupSecret = f'{setupPrefix}set aws_secret_access_key {awsSecret}'
  setupRegion = f'{setupPrefix}set default.region {region}'
  return (setupKey, setupSecret, setupRegion)
 
  

In [ ]:
def testSetUpAws():
  r = setUpAws('test', 'secret', 'testProfile')
  print(r)
testSetUpAws()

('aws configure --profile testProfile set aws_access_key_id test', 'aws configure --profile testProfile set aws_secret_access_key secret', 'aws configure --profile testProfile set default.region ap-southeast-1')


In [ ]:
#export
@beartype
def setUpAws2(awsKey:str, awsSecret:str, 
             profile:Optional[str] = None, region:str = 'ap-southeast-1')->Tuple[str,str]:
  
  configString = f'''[{profile or 'default'}]\nregion = {region}\noutput = json'''
  
  credentialString =  f'''[{profile or 'default'}]\naws_access_key_id = {awsKey}\naws_secret_access_key = {awsSecret}''' 
  return configString, credentialString

### Shield strings
shield the secrets

In [ ]:
#export
def shieldStrings(strings):
  r = ''
  for line in strings.split('\n'):
    words = line.split(' ')
    r += ' '.join(words[:2] + [f'{"*" * len(words[2])}' if len(line.split(' ')) == 3 else '']) + '\n'
  return r

In [ ]:
def testSetUpAws2():
  i,j = setUpAws2('test', 'secret', 'testProfile')
  print(shieldStrings(i))
  print(shieldStrings(j))
  
testSetUpAws2()

[testProfile] 
region = **************
output = ****

[testProfile] 
aws_access_key_id = ****
aws_secret_access_key = ******



# autosetup aws

In [ ]:
#export
@beartype
def autoSetupAws(path:str, profile:Optional[str]=None, region:str='ap-southeast-1', mockup:bool = False, password:Optional[bytes] = None):
  key, secret = loadAwsPw(path, password=password) # key and secret in bytes
  try:
    key, secret = key.decode(), secret.decode()
  except:
    print('cant decode key and secret, maybe the password is wrong')
    return
    
  config, creds = setUpAws2(awsKey=key,
                          awsSecret=secret,
                          profile=profile,
                          region=region)
  print('config is :', shieldStrings(config), shieldStrings(creds), sep='\n')
  
  
  # check if reunning in colab
  isColab =  'google.colab' in str(get_ipython())
  if isColab:
    print('Running on CoLab')
  else:
    print('Not running on colab')
    
  awsPath = f'{Path.home()}/.aws'
  print(f'awspath is {awsPath}')
    
  if isColab:
    print('creating a folder')
    try:
      os.mkdir(awsPath)
    except:
      print('path exist')
    awsPath = f'{Path.home()}/.aws'
    print('writing creds files')
    with open (f'{awsPath}/credentials', 'w') as f:
      f.write(creds)
    with open (f'{awsPath}/config', 'w') as f:
      f.write(config)
  else:
    print('not colab')

In [ ]:
def testAutoSetupAws():
  autoSetupAws(path = '/tmp/test', password=b'test')
  print('test successful')
  autoSetupAws(path = '/tmp/test', password=b'tst')
testAutoSetupAws()

loading password from /tmp/test
config is :
[default] 
region = **************
output = ****

[default] 
aws_access_key_id = ****
aws_secret_access_key = ********

Not running on colab
awspath is /home/ec2-user/.aws
not colab
test successful
loading password from /tmp/test
cant decode key and secret, maybe the password is wrong
